#Майнор ИАД. Домашнее задание 3. YOLO.

В этом задании вы напишете и обучите свой собственный YOLO детектор. Нужно будет разобраться со статьей: понять какого формата должна быть обучающая пара (x, y), как перевести лосс из математической формулы в питоновский код - ну и конечно понять и реализовать саму архитектуру модели.

Выборка на котрой мы будем обучать модель состоит из разнообразных фотографий яблок, бананов и апельсинов. Данные скачиваем [отсюда](https://drive.google.com/file/d/1d8GSfZoWbraWCSUhX78yl4CnMFYE-5n3/view?usp=sharing).

Баллы за ДЗ распределены следующим образом: 
- Выборка для YoloV1 - 2 балла
- YOLO модель - 2 балла
- YOLO Loss - 3 балла
- Вспомогательные функции - 2 балла
- Обучение и расчет метрик - 2 балла

Для построения и обучения можно использовать как pytorch, так и pytorch-lightning.

Да-да, баллов в сумме получается 11

In [ ]:
# Данная библиотека понадобится нам, чтобы обработать разметку
! pip install xmltodict pytorch-lightning

Скачаем данные

In [ ]:
!wget --quiet --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://drive.google.com/uc?export=download&id=1d8GSfZoWbraWCSUhX78yl4CnMFYE-5n3' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1d8GSfZoWbraWCSUhX78yl4CnMFYE-5n3" -O data.zip && rm -rf /tmp/cookies.txt
!unzip -q data.zip
!rm data.zip
!ls -l

total 12
drwx------ 4 root root 4096 Nov 26 08:32 data
drwxr-xr-x 3 root root 4096 Dec 10 19:25 __MACOSX
drwxr-xr-x 1 root root 4096 Dec  8 14:36 sample_data


Посмотрим как выглядит один из файлов разметки

In [ ]:
!cat data/train/apple_3.xml

## Релизуйте выборку для YoloV1 - 2 балла

In [ ]:
import os
import cv2
import json
import glob
import tqdm
import xmltodict

from IPython.core.display import struct

from typing import List

import pandas as pd
import numpy as np

import torch
import torchvision
import torchvision.transforms as transforms
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

import pytorch_lightning as pl

import albumentations as A
import albumentations.pytorch

from PIL import Image

import matplotlib.pyplot as plt

from sklearn.metrics import auc
# Добавьте необходимые вам библиотеки, если их не окажется в списке выше

In [ ]:
image_paths = glob.glob(os.path.join('data/train/','*.jpg'))

In [ ]:
fileptr = open("/content/data/train/orange_76.xml","r")

#read xml content from the file
xml_content= fileptr.read()

#change xml format to ordered dict
my_ordered_dict=xmltodict.parse(xml_content)
my_ordered_dict

{'annotation': {'folder': 'train',
  'filename': 'orange_76.jpg',
  'path': 'C:\\tensorflow1\\models\\research\\object_detection\\images\\train\\orange_76.jpg',
  'source': {'database': 'Unknown'},
  'size': {'width': '1620', 'height': '1080', 'depth': '3'},
  'segmented': '0',
  'object': [{'name': 'orange',
    'pose': 'Unspecified',
    'truncated': '1',
    'difficult': '0',
    'bndbox': {'xmin': '494', 'ymin': '1', 'xmax': '1452', 'ymax': '741'}},
   {'name': 'orange',
    'pose': 'Unspecified',
    'truncated': '1',
    'difficult': '0',
    'bndbox': {'xmin': '822',
     'ymin': '643',
     'xmax': '1620',
     'ymax': '1066'}}]}}

In [ ]:
len(my_ordered_dict['annotation']['object'])

2

In [ ]:
dic_t = xmltodict.parse(open("/content/data/train/orange_76.xml","r").read())['annotation']['object']
dic_t[0]['bndbox']['xmax']

'1452'

In [ ]:
im = np.array(Image.open('/content/data/train/mixed_8.jpg').convert("RGB"))
im.shape[0]

757

Так как в этом домашнем задании использовать аугментации для обучения __обязательно__ - советуем воспользоваться библиотекой albumentations.

Она  особенно удобна, поскольку умеет сама вычислять новые координаты bounding box'ов после трансформаций картинки. Для знакомства с этим механизмом советуем следующий [гайд](https://albumentations.ai/docs/getting_started/bounding_boxes_augmentation/). 

Вы все еще можете избрать путь torchvision.transforms, вам потребуется знакомый нам метод `__getitem__`, однако вычислять новые координаты bounding box'ов после трансформаций вам придётся вручную

__Обратите внимание__ на то, что в статье коробки предсказаний параметризуются через: _(x_center, y_center, width, height)_ (причем эти значения _относительные_), а в наших файлах - это _(x_min, y_min, x_max, y_max)_

Также, помните что модель должна предсказывать как прямоугольник с обьектом, так и вероятности каждого класса!

In [ ]:
from sklearn import preprocessing

In [ ]:
class2tag = {"apple": 1, "orange": 2, "banana": 3}

class FruitDataset(Dataset):
    def __init__(self, data_dir, transforms=None):
        self.image_paths = glob.glob( os.path.join(data_dir,'*.jpg'))
        self.enc = preprocessing.LabelEncoder()
        self.box_paths = [x+".xml" for x in [os.path.splitext(x)[0] for x in self.image_paths]]
        assert len(self.image_paths) == len(self.box_paths)
        self.transforms = transforms
        for idx in range(len(self.box_paths)):
            image = np.array(Image.open(self.image_paths[idx]).convert("RGB"))
            _,classes =  self.__get_boxes_from_xml(self.box_paths[idx],image)
            if idx == 0:
              all_classes = classes
            classes = np.array(classes)
            all_classes = np.concatenate((all_classes,classes))
        all_classes = np.unique(all_classes)
        self.enc.fit(all_classes)


    # Координаты прямоугольников советуем вернуть именно в формате (x_center, y_center, width, height)
    def __getitem__(self, idx):
        image = np.array(Image.open(self.image_paths[idx]).convert("RGB"))
        boxes, class_labels = self.__get_boxes_from_xml(self.box_paths[idx],image)
        if self.transforms is not None:
          augs = self.transforms(image=image, bboxes = boxes, class_labels=class_labels)
          image = augs['image']
          boxes = augs['bboxes']
          class_labels = augs['class_labels']
        boxes, class_labels = torch.as_tensor(boxes, dtype=torch.float64),  torch.as_tensor(self.enc.transform(class_labels),dtype=torch.int8)
        return image.unsqueeze(dim=0), (boxes, class_labels)

    def __len__(self):
        return len(self.image_paths)

    def __get_boxes_from_xml(self, xml_filename: str,image):
      """
      Метод, который считает и распарсит (с помощью xmltodict) переданный xml
      файл и вернет координаты прямоугольников обьектов на соответсвующей фотографии
      и название класса обьекта в каждом прямоугольнике

      Обратите внимание, что обьектов может быть как несколько, так и один единственный
      """
      boxes = []
      class_labels = []
      dic = xmltodict.parse(open(xml_filename,"r").read())['annotation']
      im_w = int(dic['size']['width'])
      if im_w == 0:
        im_w = image.shape[1]
      im_h = int(dic['size']['height'])
      if im_h == 0:
        im_h = image.shape[0]
      obj = dic['object']
      if isinstance(obj, dict):
        obj = [obj]
      for item in obj:
        boxes.append(self.__convert_to_yolo_box_params(
            [int(x) for x in [item['bndbox']['xmin'], item['bndbox']['ymin'], item['bndbox']['xmax'], item['bndbox']['ymax']]],
            im_w,im_h)
        ) 
        class_labels.append(item['name'])      
      return boxes,class_labels

    def __convert_to_yolo_box_params(self, box_coordinates: List[int], im_w, im_h):
      """
      Перейти от [xmin, ymin, xmax, ymax] к [x_center, y_center, width, height].
      
      Обратите внимание, что параметры [x_center, y_center, width, height] - это
      относительные значение в отрезке [0, 1]

      :param: box_coordinates - координаты коробки в формате [xmin, ymin, xmax, ymax]
      :param: im_w - ширина исходного изображения
      :param: im_h - высота исходного изображения

      :return: координаты коробки в формате [x_center, y_center, width, height]
      """
      ans = [] 
      ans.append((box_coordinates[0] + box_coordinates[2]) / 2 / im_w)# x_center  
      ans.append((box_coordinates[1] + box_coordinates[3]) / 2 / im_h)  # y_center
      ans.append((box_coordinates[2] - box_coordinates[0]) / im_w)# width  
      ans.append((box_coordinates[3] - box_coordinates[1]) / im_h)  # height 
      return ans

In [ ]:
transform = A.Compose([A.ToFloat (max_value=None, always_apply=False, p=1.0),
    A.pytorch.transforms.ToTensorV2()
                       ], 
                             bbox_params=A.BboxParams(format='yolo',
                                                      label_fields=['class_labels']))
train_dataset = FruitDataset(transforms=transform,
    data_dir="./data/train")
train_dataloader = DataLoader(dataset=train_dataset, batch_size=1)
test_dataset = FruitDataset(transforms=transform,
    data_dir="./data/test")
test_dataloader = DataLoader(dataset=test_dataset, batch_size=1)

/usr/local/lib/python3.8/dist-packages/PIL/Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [ ]:
def get_mean_and_std(dataloader):
    channels_sum, channels_squared_sum, num_batches = 0, 0, 0
    for data, _ in dataloader:
        # Mean over batch, height and width, but not over the channels
        print(data)
        channels_sum += torch.mean(data[0], dim=[0,2,3])
        channels_squared_sum += torch.mean(data[0]**2, dim=[0,2,3])
        num_batches += 1
    
    mean = channels_sum / num_batches

    # std = sqrt(E[X^2] - (E[X])^2)
    std = (channels_squared_sum / num_batches - mean ** 2) ** 0.5

    return mean, std

In [ ]:
mean, std = get_mean_and_std(train_dataloader)
mean_tr,std_tr = mean.tolist(),std.tolist()
mean, std = get_mean_and_std(test_dataloader)
mean_test,std_test = mean.tolist(),std.tolist()

In [ ]:
mean_tr

[0.8134564757347107, 0.6914446949958801, 0.5386325716972351]

In [ ]:
std_tr

[0.27581149339675903, 0.32317855954170227, 0.40659812092781067]

In [ ]:
mean_test

[0.8366211652755737, 0.7004529237747192, 0.56198650598526]

In [ ]:
std_test

[0.25586554408073425, 0.3319564163684845, 0.4067434072494507]

In [ ]:
WIDTH, HEIGHT = 448,448

train_transform = A.Compose([A.HorizontalFlip(p=0.5),
    A.ColorJitter (brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2, always_apply=False, p=0.5),
    A.GaussNoise (var_limit=(10.0, 50.0), mean=0, per_channel=True, always_apply=False, p=0.5),
    A.Normalize (mean=mean_tr, std=std_tr, max_pixel_value=255.0, always_apply=False, p=1.0),A.Resize(WIDTH, HEIGHT),
    A.pytorch.transforms.ToTensorV2(), ], 
                             bbox_params=A.BboxParams(format='yolo',
                                                      label_fields=['class_labels']))
test_transform = A.Compose([A.HorizontalFlip(p=0.5),
    A.ColorJitter (brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2, always_apply=False, p=0.5),
    A.GaussNoise (var_limit=(10.0, 50.0), mean=0, per_channel=True, always_apply=False, p=0.5),
    A.Normalize (mean=mean_test, std=std_test, max_pixel_value=255.0, always_apply=False, p=1.0),
    A.pytorch.transforms.ToTensorV2()],
                            bbox_params=A.BboxParams(format='yolo',
                                                     label_fields=['class_labels']))

In [ ]:
train_dataset = FruitDataset(
    transforms=train_transform,
    data_dir="./data/train"
    )

val_dataset = FruitDataset(
    transforms=test_transform, 
    data_dir="./data/test"
    )

# Немного проверок, чтобы убедиться в правильности направления решения
assert isinstance(train_dataset[2], tuple)
assert len(train_dataset[2]) == 2
assert isinstance(train_dataset[2][0], torch.Tensor)
assert isinstance(train_dataset[2][1], tuple)
assert len(train_dataset[2][1]) == 2
print("Тесты успешно пройдены")

Тесты успешно пройдены


In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))

In [ ]:
train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size= 16,
    num_workers = 4,
    shuffle=True, collate_fn=collate_fn)

val_dataloader = DataLoader(
    dataset=val_dataset,
    batch_size=16,
    num_workers = 4, 
    shuffle=False,
    collate_fn=collate_fn
    )

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Теперь определим функцию для рассчета Intersection Over Union по 4 углам двух прямоугольников

In [ ]:
def intersection_over_union(predicted_bbox, gt_bbox) -> float:
    """
    Intersection Over Union для двух прямоугольников

    :param: predicted_bbox - [x_min, y_min, x_max, y_max]
    :param: gt_bbox - [x_min, y_min, x_max, y_max]
    
    :return: Intersection Over Union
    """

    intersection_bbox = np.array(
        [
            max(predicted_bbox[0], gt_bbox[0]),
            max(predicted_bbox[1], gt_bbox[1]),
            min(predicted_bbox[2], gt_bbox[2]),
            min(predicted_bbox[3], gt_bbox[3]),
        ]
    )

    intersection_area = max(intersection_bbox[2] - intersection_bbox[0], 0) * max(
        intersection_bbox[3] - intersection_bbox[1], 0
    )
    area_dt = (predicted_bbox[2] - predicted_bbox[0]) * (predicted_bbox[3] - predicted_bbox[1])
    area_gt = (gt_bbox[2] - gt_bbox[0]) * (gt_bbox[3] - gt_bbox[1])

    union_area = area_dt + area_gt - intersection_area

    iou = intersection_area / union_area
    return iou

In [ ]:
x = next(iter(train_dataloader))
print(x[1])

((tensor([[0.2542, 0.4064, 0.3500, 0.5920],
        [0.6193, 0.3504, 0.3427, 0.5152]], dtype=torch.float64), tensor([2, 2], dtype=torch.int8)), (tensor([[0.5166, 0.4232, 0.3047, 0.3750],
        [0.5054, 0.5612, 0.8057, 0.2552],
        [0.5239, 0.7240, 0.8232, 0.2708]], dtype=torch.float64), tensor([2, 1, 1], dtype=torch.int8)), (tensor([[0.4725, 0.5820, 0.9283, 0.4680]], dtype=torch.float64), tensor([1], dtype=torch.int8)), (tensor([[0.4319, 0.7062, 0.6722, 0.5877],
        [0.7438, 0.4549, 0.5097, 0.5840]], dtype=torch.float64), tensor([0, 0], dtype=torch.int8)), (tensor([[0.6183, 0.6052, 0.5934, 0.6442]], dtype=torch.float64), tensor([0], dtype=torch.int8)), (tensor([[0.4142, 0.3472, 0.8108, 0.6922]], dtype=torch.float64), tensor([1], dtype=torch.int8)), (tensor([[0.4208, 0.5652, 0.6550, 0.8323]], dtype=torch.float64), tensor([2], dtype=torch.int8)), (tensor([[0.3096, 0.6196, 0.6013, 0.7608],
        [0.6936, 0.5456, 0.6051, 0.9089]], dtype=torch.float64), tensor([0, 0], dtype=torc

Теперь начинается основная часть домашнего задания: обучите модель YOLO для object detection на __обучающем__ датасете. 

 - Создайте модель и функцию ошибки YoloV1 прочитав [оригинальную статью](https://paperswithcode.com/paper/you-only-look-once-unified-real-time-object)
 - Напишите функцию обучения модели
 - Используйте аугментации

## Реализуйте Модель - 2 балла

Копировать точное количество слоев и параметры сверток необязательно. Главное - чтобы модель работала по принципу, описанному в статье и делала предсказание в представленном формате.


В качестве подсказки напомним, что выходом модели __для каждого обьекта__ должен быть тензор размера
__S * S * (B * 5 + С)__, где все параметры имеют такое же значение, как и в статье: 

- S - количество ячеек на которое разбивается изображение по вертикали/горизонтали
- В - количество предсказываемых прямоугольников в каждой ячейке
- 5 - количество параметров для определения каждого прямоугольника (x_center, y_center, width, height, confidence)
- С - количество классов (apple, banana, orange)

Таким образом, мы для каждого окна размера __S x S__ предсказываем __В__ коробо и один класс

In [ ]:
params = [ #[channels, Maxpool, kernel, padding, stride]
    [64,True,7,3,2],
    [192,True,3,1,1],
    [128,False,1,0,1],
    [256,False,3,1,1],
    [256,False,1,0,1],
    [512,True,3,1,1],
    (4,
    [[256,False,1,0,1],
    [512,False,3,1,1]]),
    [512,False,1,0,1],
    [1024,True,3,1,1],
    (2,
    [[512,False,1,0,1],
    [1024,False,3,1,1]]),
    [1024,False,3,1,1],
    [1024,False,3,1,2],
    (2,[[1024,False,3,1,1]])
]

In [ ]:
class CNNBlock(nn.Module):  # можно поменять на Lightning
    def __init__(self, in_channels, out_channels, is_max_pool:bool=False, **kwargs):
        
        super().__init__()

        self.conv = nn.Conv2d(in_channels, out_channels, **kwargs)
        self.batchnorm = nn.BatchNorm2d(out_channels)  # в статье еще не знали про батчнорм, но мы то из будущего ...
        self.leakyrelu = nn.LeakyReLU(0.1)

        self.is_maxpool = is_max_pool  # не после каждой свертки нужно делать maxpool
        self.maxpool = nn.MaxPool2d(2)
        
    def forward(self, x):
        x = self.leakyrelu(self.batchnorm(self.conv(x)))

        if self.is_maxpool:
            x = self.maxpool(x)

        return x
    
class YOLO(nn.Module):
    def __init__(self, S=7, B=2, C=3,in_channels:int = 3, architecture:list=params,**kwargs):
        """
        :param: S * S - количество ячеек на которые разбивается изображение
        :param: B - количество предсказанных прямоугольников в каждой ячейке
        :param: C - количество классов
        """
        
        super(YOLO, self).__init__()

        self.S = S
        self.B = B
        self.C = C
        self.in_channels=in_channels
        self.arch = architecture
        self.net = self._create_conv_layers()
        self.fc = self._create_fcs(**kwargs)

    def forward(self, x):
        for block in self.net:
          x= block(x)
        return self.fc(x)


    def _create_conv_layers(self):
      in_channels = self.in_channels
      conv = nn.ModuleList()
      for block in self.arch:
        if isinstance(block, list):
          conv.append(CNNBlock(in_channels,*block[0:2],kernel_size=block[2],padding=block[3],stride=block[4]))
          in_channels = block[0]
        elif isinstance(block, tuple):
          for _ in range(block[0]):
            for small_block in block[1]:
              conv.append(CNNBlock(in_channels,*small_block[0:2],kernel_size=small_block[2],padding=small_block[3],stride=small_block[4]))
              in_channels=small_block[0]
        else:
          raise TypeError("architecture can include only types: 'list' or 'tuple'")
      return conv


    def _create_fcs(self,p = 0.1):
      return nn.Sequential(
            nn.Flatten(),
            nn.Linear(1024 * self.S * self.S, 4096),
            nn.Dropout(p = p),
            nn.LeakyReLU(0.1),
            nn.Linear(4096, self.S * self.S * (self.C + self.B * 5)),
        )

temp_model = YOLO()
expected_output_shape = temp_model.S * temp_model.S * (5 * temp_model.B + temp_model.C)

assert temp_model(train_dataset[0][0]).reshape(-1).shape[0] == expected_output_shape
temp_model(train_dataset[0][0]).size()

torch.Size([1, 637])

## Реализуйте YoloLoss - 3 балла

In [ ]:
x = torch.empty(2,2)
x[0][0],x[0][1],x[1][0],x[1][1] = 1,2,-1,-2
print(x)
print(x[0])
torch.max(x,dim=1)

tensor([[ 1.,  2.],
        [-1., -2.]])
tensor([1., 2.])


torch.return_types.max(
values=tensor([ 2., -1.]),
indices=tensor([1, 0]))

In [ ]:
y = torch.tensor([[[1,2],[3,4]],[[-1,8],[-3,-4]]])
z = torch.tensor([[-1,8],[-3,-4]])
yz = torch.cat([y.unsqueeze(0),z.unsqueeze(0)],dim=0)
print(yz)
torch.argmax(yz,dim=0)

tensor([[[ 1,  2],
         [ 3,  4]],

        [[-1,  8],
         [-3, -4]]])


tensor([[0, 1],
        [0, 0]])

In [ ]:
y = torch.FloatTensor([[[2,2],[3,4]],[[-1,8],[-3,-4]],[[-1,8],[-3,-4]]])
I = torch.FloatTensor([[1,0],[0,1]])
o = torch.FloatTensor([[3,2],[1,1]])
torch.flatten(y)
I*y
#y.size()

tensor([[[ 2.,  0.],
         [ 0.,  4.]],

        [[-1.,  0.],
         [-0., -4.]],

        [[-1.,  0.],
         [-0., -4.]]])

In [ ]:
z = 2*torch.FloatTensor([[[1,2],[3,4]],[[-1,8],[-3,-4]]])
torch.flatten(z)

tensor([ 2.,  4.,  6.,  8., -2., 16., -6., -8.])

In [ ]:
k = torch.flatten(z)
float(k[0])

2.0

In [ ]:
mse = nn.MSELoss(reduction="sum")
mse(torch.flatten(y),torch.flatten(z))

tensor(120.)

In [ ]:
loss=0
y_0 = torch.flatten(y)
z_0 = torch.flatten(z)
for i in range(len(y_0)):
  loss += (float(y_0[i])-float(z_0[i]))**2
loss

120.0

In [ ]:
from itertools import product

In [ ]:
def get_target(boxes, labels,S=7, B=2, C=3):
  label_matrix = torch.zeros((C + 5,S, S ))
  for box,i in enumerate(boxes):
    x,y,w,h = box.tolist()
    class_label = labels[i]
    class_label = int(class_label)

    i, j = int(S * y), int(S * x)
    x_cell, y_cell = S * x - j, S * y - i
    width_cell, height_cell = (w * S, h * S)

    if label_matrix[C,i, j] == 0:
        label_matrix[C,i, j] = 1
        box_coordinates = torch.tensor(
            [x_cell, y_cell, width_cell, height_cell]
        )

        label_matrix[C+1:,i, j ] = box_coordinates

        # Set one hot encoding for class_label
        label_matrix[class_label,i, j ] = 1

        return label_matrix

In [ ]:
def flat

In [ ]:
class YoloLoss(nn.Module):
    def __init__(self, S=7, B=2, C=3):
        """
        :param: S * S - количество ячеек на которые разбивается изображение
        :param: B - количество предсказанных прямоугольников в каждой ячейке
        :param: C - количество классов
        """
        
        super().__init__()
        self.mse = nn.MSELoss(reduction="sum")

        self.S = S
        self.B = B
        self.C = C

        self.lambda_noobj = 0.5
        self.lambda_coord = 5

    def forward(self, predictions, target):
        self.pred = predictions.reshape(self.C+5*self.B,self.S,self.S)
        loss = 0
        iou = torch.empty((self.B,self.S,self.S))
        box_pred = torch.empty((4,self.S,self.S))
        ######################################
        ###           Cord Loss            ###
        ######################################
        for x,y,n in product(range(self.S),range(self.S),range(self.B)):
          iou[n,x,y] = intersection_over_union(self.pred[self.C+5*n+1:self.C+5*n+5,x,y],self.target[self.C+1:self.C+5,...])
        best_boxes = torch.argmax(iou,dim=0)
        I_obj = target[self.C,...]
        for x,y in product(range(self.S),range(self.S)):
          box_pred[:,x,y]=self.pred[self.C+5*int(best_boxes[x,y])+1:self.C+5*int(best_boxes[x,y])+5,x, y]
        box_pred = I_obj*box_pred
        box_targ = I_obj * target[self.C+1:self.C+5,...]
        box_pred[2:4,...],box_targ[2:4,...] = torch.sqrt(torch.abs(box_pred[2:4,...])),torch.sqrt(torch.abs(box_targ[2:4,...])) 
        loss +=self.lambda_coord*self.mse(torch.flatten(box_pred),torch.flatten(box_targ))
        ######################################
        ###            Obj Loss            ###
        ######################################
        obj_pred = torch.empty((self.S,self.S))
        for x,y in product(range(self.S),range(self.S)):
          obj_pred[x,y]=self.pred[self.C+5*int(best_boxes[x,y]),x, y]
        obj_pred = I_obj*obj_pred
        obj_targ = I_obj* target[self.C,...]
        loss += self.mse(torch.flatten(obj_pred),torch.flatten(obj_targ))
        ######################################
        ###           No Obj Loss          ###
        ######################################
        I_noobj = 1-I_obj
        noobj_targ = I_noobj*target[self.C,...]
        for i in range(self.B):
          noobj_pred = I_noobj*self.pred[self.C+5*i,x, y]
          loss += self.lambda_noobj*self.mse(torch.flatten(noobj_pred),torch.flatten(noobj_targ))
        ######################################
        ###           Class Loss           ###
        ######################################
        loss += self.mse(
            torch.flatten(I_obj * self.pred[:self.C,...]),
            torch.flatten(I_obj * target[:self.C,... ] ),
        )
        return loss

In [ ]:
c=128
x = torch.zeros((3,c,c))
for i in range(c):
  x[1,c-1,i]=1
  x[2,c-1,i]=1
  x[0,c-1,i]=1
  x[1,i,c-1]=1
  x[2,i,c-1]=1
  x[0,i,c-1]=1
#x.size()
weights = torch.ones((3,3,3,3))
p = torch.nn.Conv2d(3,3,kernel_size=3,stride=2)
with torch.no_grad():
    p.weight = nn.Parameter(weights)
y = p(x)
print(x)
print(y)
y.size()

tensor([[[0., 0., 0.,  ..., 0., 0., 1.],
         [0., 0., 0.,  ..., 0., 0., 1.],
         [0., 0., 0.,  ..., 0., 0., 1.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 1.],
         [0., 0., 0.,  ..., 0., 0., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]],

        [[0., 0., 0.,  ..., 0., 0., 1.],
         [0., 0., 0.,  ..., 0., 0., 1.],
         [0., 0., 0.,  ..., 0., 0., 1.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 1.],
         [0., 0., 0.,  ..., 0., 0., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]],

        [[0., 0., 0.,  ..., 0., 0., 1.],
         [0., 0., 0.,  ..., 0., 0., 1.],
         [0., 0., 0.,  ..., 0., 0., 1.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 1.],
         [0., 0., 0.,  ..., 0., 0., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]]])
tensor([[[-0.0875, -0.0875, -0.0875,  ..., -0.0875, -0.0875, -0.0875],
         [-0.0875, -0.0875, -0.0875,  ..., -0.0875, -0.0875, -0.0875],
         [-0.0875, -0.0875, -0.0875,  ..., -0.0875, -0.0875, -0.0875],
 

torch.Size([3, 63, 63])

In [ ]:
m=nn.MaxPool2d(2)
z = m(y)
print(z)
z.size()

tensor([[[-0.0875, -0.0875, -0.0875,  ..., -0.0875, -0.0875, -0.0875],
         [-0.0875, -0.0875, -0.0875,  ..., -0.0875, -0.0875, -0.0875],
         [-0.0875, -0.0875, -0.0875,  ..., -0.0875, -0.0875, -0.0875],
         ...,
         [-0.0875, -0.0875, -0.0875,  ..., -0.0875, -0.0875, -0.0875],
         [-0.0875, -0.0875, -0.0875,  ..., -0.0875, -0.0875, -0.0875],
         [-0.0875, -0.0875, -0.0875,  ..., -0.0875, -0.0875, -0.0875]],

        [[ 0.0751,  0.0751,  0.0751,  ...,  0.0751,  0.0751,  0.0751],
         [ 0.0751,  0.0751,  0.0751,  ...,  0.0751,  0.0751,  0.0751],
         [ 0.0751,  0.0751,  0.0751,  ...,  0.0751,  0.0751,  0.0751],
         ...,
         [ 0.0751,  0.0751,  0.0751,  ...,  0.0751,  0.0751,  0.0751],
         [ 0.0751,  0.0751,  0.0751,  ...,  0.0751,  0.0751,  0.0751],
         [ 0.0751,  0.0751,  0.0751,  ...,  0.0751,  0.0751,  0.0751]],

        [[-0.1422, -0.1422, -0.1422,  ..., -0.1422, -0.1422, -0.1422],
         [-0.1422, -0.1422, -0.1422,  ..., -0

torch.Size([3, 31, 31])

In [ ]:
weights1 = torch.ones((3,3,5,5))
k = torch.nn.Conv2d(3,3,kernel_size=5,stride=2)
with torch.no_grad():
    k.weight = nn.Parameter(weights1)
r = k(z)
#print(x)
print(r)
r.size()

tensor([[[-3.7719, -3.7719, -3.7719, -3.7719, -3.7719, -3.7719, -3.7719,
          -3.7719, -3.7719, -3.7719, -3.7719, -3.7719, -3.7719, -3.7719],
         [-3.7719, -3.7719, -3.7719, -3.7719, -3.7719, -3.7719, -3.7719,
          -3.7719, -3.7719, -3.7719, -3.7719, -3.7719, -3.7719, -3.7719],
         [-3.7719, -3.7719, -3.7719, -3.7719, -3.7719, -3.7719, -3.7719,
          -3.7719, -3.7719, -3.7719, -3.7719, -3.7719, -3.7719, -3.7719],
         [-3.7719, -3.7719, -3.7719, -3.7719, -3.7719, -3.7719, -3.7719,
          -3.7719, -3.7719, -3.7719, -3.7719, -3.7719, -3.7719, -3.7719],
         [-3.7719, -3.7719, -3.7719, -3.7719, -3.7719, -3.7719, -3.7719,
          -3.7719, -3.7719, -3.7719, -3.7719, -3.7719, -3.7719, -3.7719],
         [-3.7719, -3.7719, -3.7719, -3.7719, -3.7719, -3.7719, -3.7719,
          -3.7719, -3.7719, -3.7719, -3.7719, -3.7719, -3.7719, -3.7719],
         [-3.7719, -3.7719, -3.7719, -3.7719, -3.7719, -3.7719, -3.7719,
          -3.7719, -3.7719, -3.7719, -3.7719,

torch.Size([3, 14, 14])

In [ ]:
float(x[1,0,0])

-5978139209498624.0

## Реализуйте дополнительные функции из статьи - 2 балла

In [ ]:
def non_max_suppression(bboxes, iou_threshold, threshold):
    ## YOUR CODE
    pass

def mean_average_precision(pred_boxes, true_boxes, iou_threshold=0.5):
    ## YOUR CODE
    pass

def get_bound_boxes(loader, model, iou_threshold=.5, threshold=.4):
    ## YOUR CODE
    return all_pred_boxes, all_true_boxes

## Обучите модель и посчитайте метрики для задачи детекции - 2 балла 

Несмотря на то, что в этом блоке ничего сильно нового для вас не ожидается и за него формально дается лишь два балла - провести обучение очень важно для понимания того, насколько правильно реализована ваша модель и лосс.

В процессе обучения будет видно все ли размерности совпадают, падает ли лосс и растут ли метрики целевой задачи, поэтому на практике этот пункт гораздо оказывается гораздо важнее.

In [ ]:
class YOLOLearner(pl.LightningModule):
    def __init__(self) -> None:
        super().__init__()

        self.model = YOLO()
        self.loss = YoloLoss()
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=1e-3)

    def forward(self, x) -> torch.Tensor:
        return self.model(x)

    def configure_optimizers(self):
        return self.optimizer

    def training_step(self, train_batch, batch_idx) -> torch.Tensor:
        ## YOUR CODE
        pass

    def validation_step(self, val_batch, batch_idx) -> None:
        ## YOUR CODE
        pass

In [ ]:
model = # YOUR CODE
n_epochs = # YOUR CODE

yolo_learner = YOLOLearner(...)  ## YOUR CODE

device = "gpu" if torch.cuda.is_available() else "cpu"
trainer = pl.Trainer(accelerator=device, max_epochs=n_epochs)

trainer.fit(yolo_learner, train_dataloader, val_dataloader)

## Посчитайте метрики задачи детекции на валидационной выборке

Попробуйте понять насколько хороши ваши показатели. Если числа кажутся подозрительно низкими - возможно вам стоит перепроверить ваше решение. 

In [ ]:
## YOUR CODE

## Визуализируйте предсказанные bounding box'ы для любых пяти картинок из __валидационного__ датасета.

In [ ]:
image, targets = next(iter(val_dataset))
preds = ## YOUR CODE

In [ ]:
from PIL import ImageDraw

image = torchvision.transform.ToPILImage()(image)
draw = ImageDraw.Draw(image)

for box in targets[0]:
    ## YOUR CODE
    draw.rectangle([(box[0], box[1]), (box[2], box[3])])

for box in preds[0]:
    ## YOUR CODE
    draw.rectangle([(box[0], box[1]), (box[2], box[3])], outline='red')
image